# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 7 2023 8:30AM,258777,10,0086483485,ISDIN Corporation,Released
1,Mar 7 2023 8:30AM,258777,10,0086483484,ISDIN Corporation,Released
2,Mar 7 2023 8:30AM,258777,10,0086483820,ISDIN Corporation,Released
3,Mar 7 2023 8:30AM,258777,10,0086484265,ISDIN Corporation,Released
4,Mar 7 2023 8:30AM,258777,10,0086484266,ISDIN Corporation,Released
5,Mar 7 2023 8:30AM,258777,10,0086484306,ISDIN Corporation,Released
6,Mar 7 2023 8:30AM,258777,10,0086484314,ISDIN Corporation,Released
7,Mar 7 2023 8:30AM,258777,10,0086484313,ISDIN Corporation,Released
8,Mar 7 2023 8:30AM,258777,10,0086484323,ISDIN Corporation,Released
9,Mar 7 2023 8:30AM,258777,10,0086483821,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
18,258773,Released,3
19,258774,Executing,3
20,258774,Released,1
21,258777,Released,15
22,258778,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
258771,NaN,3.0
258773,NaN,3.0
258774,3.0,1.0
258777,NaN,15.0
258778,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
258691,33.0,2.0
258715,0.0,1.0
258718,0.0,2.0
258722,0.0,1.0
258741,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
258691,33,2
258715,0,1
258718,0,2
258722,0,1
258741,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,258691,33,2
1,258715,0,1
2,258718,0,2
3,258722,0,1
4,258741,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,258691,33,2
1,258715,,1
2,258718,,2
3,258722,,1
4,258741,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 7 2023 8:30AM,258777,10,ISDIN Corporation
15,Mar 7 2023 8:30AM,258768,10,Ivaoes Animal Health
16,Mar 7 2023 8:30AM,258771,10,"Nextsource Biotechnology, LLC"
19,Mar 7 2023 8:30AM,258767,10,"Methapharm, Inc."
22,Mar 7 2023 8:30AM,258769,10,"Methapharm, Inc."
24,Mar 7 2023 8:30AM,258766,10,Yusen – 3D Matrix
25,Mar 7 2023 8:30AM,258773,10,Bio-PRF
28,Mar 6 2023 4:23PM,258778,16,Sartorius Lab Products and Service
29,Mar 6 2023 3:58PM,258774,16,Sartorius Lab Products and Service
33,Mar 6 2023 2:46PM,258764,15,Lonza


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Mar 7 2023 8:30AM,258777,10,ISDIN Corporation,,15
1,Mar 7 2023 8:30AM,258768,10,Ivaoes Animal Health,,1
2,Mar 7 2023 8:30AM,258771,10,"Nextsource Biotechnology, LLC",,3
3,Mar 7 2023 8:30AM,258767,10,"Methapharm, Inc.",,3
4,Mar 7 2023 8:30AM,258769,10,"Methapharm, Inc.",,2
5,Mar 7 2023 8:30AM,258766,10,Yusen – 3D Matrix,1,
6,Mar 7 2023 8:30AM,258773,10,Bio-PRF,,3
7,Mar 6 2023 4:23PM,258778,16,Sartorius Lab Products and Service,,1
8,Mar 6 2023 3:58PM,258774,16,Sartorius Lab Products and Service,3,1
9,Mar 6 2023 2:46PM,258764,15,Lonza,1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 7 2023 8:30AM,258777,10,ISDIN Corporation,15,
1,Mar 7 2023 8:30AM,258768,10,Ivaoes Animal Health,1,
2,Mar 7 2023 8:30AM,258771,10,"Nextsource Biotechnology, LLC",3,
3,Mar 7 2023 8:30AM,258767,10,"Methapharm, Inc.",3,
4,Mar 7 2023 8:30AM,258769,10,"Methapharm, Inc.",2,
5,Mar 7 2023 8:30AM,258766,10,Yusen – 3D Matrix,,1
6,Mar 7 2023 8:30AM,258773,10,Bio-PRF,3,
7,Mar 6 2023 4:23PM,258778,16,Sartorius Lab Products and Service,1,
8,Mar 6 2023 3:58PM,258774,16,Sartorius Lab Products and Service,1,3
9,Mar 6 2023 2:46PM,258764,15,Lonza,,1


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 7 2023 8:30AM,258777,10,ISDIN Corporation,15,
1,Mar 7 2023 8:30AM,258768,10,Ivaoes Animal Health,1,
2,Mar 7 2023 8:30AM,258771,10,"Nextsource Biotechnology, LLC",3,
3,Mar 7 2023 8:30AM,258767,10,"Methapharm, Inc.",3,
4,Mar 7 2023 8:30AM,258769,10,"Methapharm, Inc.",2,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Mar 7 2023 8:30AM,258777,10,ISDIN Corporation,15.0,NaN
1,Mar 7 2023 8:30AM,258768,10,Ivaoes Animal Health,1.0,NaN
2,Mar 7 2023 8:30AM,258771,10,"Nextsource Biotechnology, LLC",3.0,NaN
3,Mar 7 2023 8:30AM,258767,10,"Methapharm, Inc.",3.0,NaN
4,Mar 7 2023 8:30AM,258769,10,"Methapharm, Inc.",2.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 7 2023 8:30AM,258777,10,ISDIN Corporation,15.0,0.0
1,Mar 7 2023 8:30AM,258768,10,Ivaoes Animal Health,1.0,0.0
2,Mar 7 2023 8:30AM,258771,10,"Nextsource Biotechnology, LLC",3.0,0.0
3,Mar 7 2023 8:30AM,258767,10,"Methapharm, Inc.",3.0,0.0
4,Mar 7 2023 8:30AM,258769,10,"Methapharm, Inc.",2.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3622583,39.0,11.0
15,517455,2.0,34.0
16,776302,3.0,3.0
20,258722,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3622583,39.0,11.0
1,15,517455,2.0,34.0
2,16,776302,3.0,3.0
3,20,258722,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,39.0,11.0
1,15,2.0,34.0
2,16,3.0,3.0
3,20,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,39.0
1,15,Released,2.0
2,16,Released,3.0
3,20,Released,1.0
4,10,Executing,11.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,20
Status,,,,
Executing,11.0,34.0,3.0,0.0
Released,39.0,2.0,3.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,20
0,Executing,11.0,34.0,3.0,0.0
1,Released,39.0,2.0,3.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,20
0,Executing,11.0,34.0,3.0,0.0
1,Released,39.0,2.0,3.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()